In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [2]:
?isposdef

search: isposdef isposdef!



```
isposdef(A) -> Bool
```

Test whether a matrix is positive definite (and Hermitian) by trying to perform a Cholesky factorization of `A`. See also [`isposdef!`](@ref)

# Examples

```jldoctest
julia> A = [1 2; 2 50]
2×2 Array{Int64,2}:
 1   2
 2  50

julia> isposdef(A)
true
```


In [4]:
a = randn(20,100)
A = a *a'

20×20 Array{Float64,2}:
 105.679       8.73631    -6.73281   …   -8.41532    13.0816       4.19571
   8.73631    89.7636     -9.57597       -0.665961   -8.11058      6.88277
  -6.73281    -9.57597    94.0622        -2.84336     6.86944     -5.60848
   1.92407     0.711452   -5.87176        0.842033    8.18098      2.3827
   2.89166    11.3947     -7.13341       -5.88191     3.448        9.34898
  -7.65365    -2.92257     5.0551    …   23.3747     -1.89557     19.4441
  -0.794266    5.55923   -12.7118        13.0472      4.10618     14.3769
  -1.34444    -6.35108     6.30012      -25.0826     20.8191     -14.0837
 -12.1415    -15.1242     14.0347        -1.19628   -12.9653       8.80836
  -4.84596    -1.23181   -15.8862         3.21458     3.691       10.3068
   3.676      14.4024      0.680349  …   12.243      -4.31091     -8.88414
  12.5466     -4.39046     3.37474      -11.5061     -3.37708    -11.6271
  -5.66285    -1.29677    10.1916         3.20392    16.234        3.42874
   7.93

In [10]:
issuccess(cholesky(Hermitian(A); check = false))

true

In [14]:
fieldnames(typeof(cholesky(Hermitian(A))))

(:factors, :uplo, :info)